In [185]:
import ira
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, GridSearchCV, GroupKFold
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline

%alphalab dark

from alpha.utils.data_helpers import FriendlyFxLoader, make_forward_returns_matrix

from qlearn.core.pickers import *
from qlearn.core.forward_returns import ForwardReturns, ForwardReturnsDirection, ForwardReturns3Classes
from qlearn.core.base import BasicMarketEstimator, MarketDataComposer
from qlearn.core.utils import crossup, crossdown, permutate_params

In [186]:
fxl = FriendlyFxLoader('dukas @ 5Min')
data = fxl.ohlcs(['EURUSD', 'GBPUSD', 'USDCHF'])

In [187]:
class CrossingMovings(BasicMarketEstimator):
    def __init__(self, fperiod, speriod, ma='sma'):
        self.fperiod = fperiod
        self.speriod = speriod
        self.ma = ma

    def fit(self, X, y, **fit_params):
        return self

    def predict(self, X):
        meta = self.metadata()
        f = smooth(X[meta.column], self.ma, self.fperiod)
        s = smooth(X[meta.column], self.ma, self.speriod)
        sigs = pd.Series(0, X.index)
        sigs.loc[crossup(f, s)] = +1
        sigs.loc[crossdown(f, s)] = -1
        return sigs.astype(int)

In [188]:
x1 = CrossingMovings(5, 25)
s1 = x1.predict(data['GBPUSD'])

In [22]:
g1 = GridSearchCV(
    cv=TimeSeriesSplit(3),
    estimator = x1.as_classifier(),
    param_grid=permutate_params({ 
        'fperiod': np.arange(7, 50).tolist(), 
        'speriod': np.arange(7, 50).tolist(), 
    }, conditions=lambda fperiod, speriod: fperiod < speriod),
)

mds = MarketDataComposer(g1, 
                         SingleInstrumentPicker(tz='EET'), 
                         ForwardReturns3Classes(horizon=1, tick_size=1e-5, tick_tolerance=25), 
                         'close', '5Min', debug=False
                        )

In [189]:
b1 = mds.fit(data, None)

In [192]:
b1.best_params_

{'EURUSD': {'fperiod': 18, 'speriod': 49},
 'GBPUSD': {'fperiod': 17, 'speriod': 49},
 'USDCHF': {'fperiod': 19, 'speriod': 49}}

In [193]:
b1.best_score_

{'EURUSD': 0.707151951802739,
 'GBPUSD': 0.5918037751085091,
 'USDCHF': 0.748780380495405}

In [217]:
signals = b1.predict(data).replace({0: np.nan}).EURUSD.dropna()
# signals = b1.predict(data).EURUSD

<hr/>

In [275]:
class FixedTracker(Tracker):
    def __init__(self, size):
        self.size = size
        
    def track(self, instrument, time, signal, quote, position, equity):
        return self.size * signal

In [ ]:
backtest(b1, data, FixedTracker(size=10000))

In [256]:
%run ../tracking/processors.py

# class BasicTracker(Tracker):

#     def track(self, instrument, signals, prices, execution_price_name='close'):
#         print(f'--(N signals : {len(signals)})----------------------------------------')
#         processor = GeneralSignalProcessor(instrument, prices, execution_price_name, True)
#         positions = processor.process(self, signals)
#         # TODO: .....
#         print(positions.head())
#         print('------------------------------------------')


# class SimpleSignalTracker(BasicTracker):

#     def __init__(self, fixed_size=1):
#         self.fixed_size = fixed_size

#     def track_signal(self, service: TradingService, signal_time, signal, signal_price, times, data):
# #         print(type(signal_time), type(signal), type(times))
# #         raise ValueError('!!!!!!!')
#         service.trade(signal_time, self.fixed_size * signal)
#         return signal_time

In [257]:
%%time
t0 = SimpleSignalTracker(10000)
t0.track('EURUSD', signals, data['EURUSD'])

--(N signals : 13655)----------------------------------------
                      EURUSD
2014-01-02 02:59:59  10000.0
2014-01-02 04:39:59 -10000.0
2014-01-02 16:34:59  10000.0
2014-01-02 19:54:59 -10000.0
2014-01-02 22:04:59  10000.0
------------------------------------------
CPU times: user 5.26 s, sys: 0 ns, total: 5.26 s
Wall time: 5.25 s


In [233]:
idx1 = data['EURUSD'].index
idx2 = data['EURUSD'].index.to_numpy()

In [236]:
idx1.get_loc(signals.index[101])

3740

In [271]:
%%time
s2 = []
for t in signals.index:
    s2.append(np.argwhere(idx2==t)[0][0])

CPU times: user 6.7 s, sys: 0 ns, total: 6.7 s
Wall time: 6.7 s


In [267]:
%%time
s1 = []
for t in signals.index:
    s1.append(idx1.get_loc(t))

CPU times: user 103 ms, sys: 18 µs, total: 103 ms
Wall time: 102 ms


In [272]:
s1 == s2

True

In [270]:
np.argwhere(idx2==signals.index.to_numpy()[101])[0][0]

3740

In [ ]:
b1.fitted_predictors_['EURUSD'].meta()